In [144]:
import polars as pl
import os
import cynde.functional as cf

In [19]:
cynde_base = r"C:\Users\Tommaso\Documents\Dev\Cynde"
cynde_data = os.path.join(cynde_base, "data_processed")

In [142]:
joined_predictions = "joined_2024-03-07_21-01-31_tggate-replica.parquet"
results_ = "results_2024-03-07_21-01-31_tggate-replica.parquet"
results_url = os.path.join(cynde_data, results_)
joined_predictions_url = os.path.join(cynde_data, joined_predictions)
pred_dfs = "predictions_2024-03-07_21-01-31_tggate-replica.parquet"
pred_dfs_url = os.path.join(cynde_data, pred_dfs)


In [145]:
df_res = pl.read_parquet(results_url)
cf.results_summary(df_res)

classifier,classifier_hp,input_features_name,mcc_train,mcc_val,mcc_test,accuracy_train,accuracy_val,accuracy_test,n
str,str,str,f64,f64,f64,f64,f64,f64,u32
"""RandomForest""","""n_estimators_1…","""COMPOUND_NAME_…",1.0,0.674853,0.357352,1.0,0.837222,0.676253,1250


In [21]:
print(joined_predictions_url)

C:\Users\Tommaso\Documents\Dev\Cynde\data_processed\joined_2024-03-07_21-01-31_tggate-replica.parquet


In [22]:
df = pl.read_parquet(joined_predictions_url)
pred_df = pl.read_parquet(pred_dfs_url)
print(df.shape)
print(pred_df.shape)

(1674, 2608)
(1674, 2555)


In [23]:
from cynde.functional.results import get_predictions, get_all_predictions_by_inputs_model

In [24]:
models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5}]}
inputs =[{"numerical":["conversations_text-embedding-3-small_embedding"]},
            {"numerical":["conversations_text-embedding-3-large_embedding"]},
            {"numerical":["conversations_text-embedding-3-small_embedding","conversations_text-embedding-3-large_embedding"]}]

In [25]:
models_dict = {"RandomForest": [{"n_estimators": 1000, "max_depth": 15}]}
inputs =[{"numerical":['COMPOUND_NAME_text-embedding-3-large_embeddings',
            'SACRIFICE_PERIOD_text-embedding-3-large_embeddings',
            'DOSE_LEVEL_text-embedding-3-large_embeddings',]},
        ]
# models_dict = {"RandomForest": [{"n_estimators": 10, "max_depth": 5}]}
# inputs = [inputs[0]]

# Call the train_nested_cv_from_np function with the required arguments


In [26]:
r_out = 10
k_out = 5
r_in = 5
k_in = 5
cv_type=("purged","stratified")

In [27]:
pred_cols = [col for col in pred_df.columns if "pred" in col]

In [28]:
outs = get_predictions(df,
                       cv_type=cv_type,
                          models=models_dict,
                          group_outer=["COMPOUND_NAME"],
                            inputs=inputs,
                            r_outer=r_out,
                            k_outer=k_out,
                            group_inner=["DOSE_LEVEL","SACRIFICE_PERIOD","target"],
                            r_inner=r_in,
                            k_inner=k_in,)                 

In [44]:
predictions_by_model = get_all_predictions_by_inputs_model(df,
                       cv_type=cv_type,
                          models=models_dict,
                          group_outer=["COMPOUND_NAME"],
                            inputs=inputs,
                            r_outer=r_out,
                            k_outer=k_out,
                            group_inner=["DOSE_LEVEL","SACRIFICE_PERIOD","target"],
                            r_inner=r_in,
                            k_inner=k_in,)               

In [45]:
for (input_name,model,hp_name,pred_cols_by_model) in predictions_by_model:
    folds= [x[0] for x in pred_cols_by_model]
    preds = [x[1] for x in pred_cols_by_model]
    df_temp = df.select(pl.col(["cv_index","target","COMPOUND_NAME","SACRIFICE_PERIOD","DOSE_LEVEL"]+folds+preds))

In [123]:
all_preds = []

for i in df["cv_index"]:
    preds_i = []
    leng = 0
    for fold, pred in pred_cols_by_model:
        if df[fold][i] == "test":
            preds_i.append(df[pred][i])
            leng += 1
    all_preds.append(preds_i)

target_as_list_list = []
for target in df["target"]:
    target_as_list = [target]*leng
    target_as_list_list.append(target_as_list)

df_preds = pl.DataFrame({"cv_index":df["cv_index"],"target": df["target"], "preds":all_preds, "target_as_list":target_as_list_list})


In [164]:
df_jj =  df.select(pl.col(["cv_index","target","COMPOUND_NAME","SACRIFICE_PERIOD","DOSE_LEVEL"])).join(df_preds.with_columns(pl.col("preds").list.len().alias("len"),(pl.col("preds").list.sum().alias("sum"))), on=["cv_index","target"],how="left")
dfjj_grouped_compound = df_jj.group_by("COMPOUND_NAME").agg(pl.col("target").mean().alias("avg_target"), pl.concat_list(pl.col("preds")).flatten(),pl.concat_list(pl.col("target_as_list")).flatten())

In [165]:
dfjj_grouped_compound

COMPOUND_NAME,avg_target,preds,target_as_list
str,f64,list[i64],list[i64]
"""fenofibrate""",0.75,"[1, 1, … 1]","[1, 1, … 1]"
"""ketoconazole""",0.5,"[0, 0, … 0]","[0, 0, … 0]"
"""dantrolene""",0.833333,"[1, 1, … 1]","[1, 1, … 1]"
"""glibenclamide""",0.666667,"[1, 1, … 0]","[1, 1, … 1]"
"""cisplatin""",0.5,"[1, 1, … 0]","[1, 1, … 0]"
"""thioridazine""",0.0,"[0, 0, … 0]","[0, 0, … 0]"
"""carbamazepine""",0.833333,"[0, 0, … 1]","[1, 1, … 1]"
"""meloxicam""",0.636364,"[1, 1, … 1]","[1, 1, … 1]"
"""caffeine""",0.083333,"[1, 1, … 0]","[0, 0, … 0]"


In [132]:
from sklearn.metrics import accuracy_score,  matthews_corrcoef

In [166]:
preds_list = dfjj_grouped_compound["preds"]
target_list = dfjj_grouped_compound["target_as_list"]
accs = []
mccs = []
for preds, target in zip(preds_list, target_list):
    acc = accuracy_score(target, preds)
    mcc = matthews_corrcoef(target, preds)
    accs.append(acc)
    mccs.append(mcc)
df_mccacc = pl.DataFrame({"COMPOUND_NAME":dfjj_grouped_compound["COMPOUND_NAME"],"acc":accs,"mcc":mccs})



In [170]:
dfjj_grouped_compound_res = dfjj_grouped_compound.join(df_mccacc, on="COMPOUND_NAME", how="left").sort("acc")
compounds = dfjj_grouped_compound_res["COMPOUND_NAME"]
avg_targets = dfjj_grouped_compound_res["avg_target"]
accs = dfjj_grouped_compound_res["acc"]
mccs = dfjj_grouped_compound_res["mcc"]
dfjj_grouped_compound_res

COMPOUND_NAME,avg_target,preds,target_as_list,acc,mcc
str,f64,list[i64],list[i64],f64,f64
"""fluoxetine hyd…",0.666667,"[0, 0, … 1]","[0, 0, … 0]",0.358667,-0.210812
"""labetalol""",0.166667,"[1, 1, … 1]","[0, 0, … 0]",0.397333,-0.393205
"""acarbose""",0.75,"[0, 0, … 0]","[1, 1, … 1]",0.403,-0.206023
"""nicotinic acid…",0.0,"[0, 0, … 0]","[0, 0, … 0]",0.411,0.0
"""ajmaline""",1.0,"[1, 1, … 0]","[1, 1, … 1]",0.411667,0.0
"""carboplatin""",0.083333,"[1, 1, … 1]","[0, 0, … 0]",0.412,0.210966
"""ranitidine""",0.166667,"[0, 0, … 0]","[0, 0, … 0]",0.418667,-0.376412
"""benziodarone""",1.0,"[1, 0, … 1]","[1, 1, … 1]",0.448667,0.0
"""butylated hydr…",0.0,"[1, 1, … 0]","[0, 0, … 0]",0.451,0.0


In [169]:
for comp, avg_target, acc , mcc in zip(compounds,avg_targets, accs, mccs):
    print(f"Compound: {comp}, avg_lesions {avg_target}" )#- Acc: {acc} - MCC: {mcc}")

Compound: fluoxetine hydrochloride, avg_lesions 0.6666666666666666
Compound: labetalol, avg_lesions 0.16666666666666666
Compound: acarbose, avg_lesions 0.75
Compound: nicotinic acid, avg_lesions 0.0
Compound: ajmaline, avg_lesions 1.0
Compound: carboplatin, avg_lesions 0.08333333333333333
Compound: ranitidine, avg_lesions 0.16666666666666666
Compound: benziodarone, avg_lesions 1.0
Compound: butylated hydroxyanisole, avg_lesions 0.0
Compound: danazol, avg_lesions 0.0
Compound: omeprazole, avg_lesions 1.0
Compound: WY-14643, avg_lesions 1.0
Compound: chlormadinone, avg_lesions 0.0
Compound: bendazac, avg_lesions 1.0
Compound: simvastatin, avg_lesions 1.0
Compound: griseofulvin, avg_lesions 0.8333333333333334
Compound: etoposide, avg_lesions 1.0
Compound: nimesulide, avg_lesions 1.0
Compound: diclofenac, avg_lesions 0.5
Compound: 2,4-dinitrophenol, avg_lesions 0.0
Compound: tiopronin, avg_lesions 0.0
Compound: famotidine, avg_lesions 0.0
Compound: pemoline, avg_lesions 0.4166666666666667


In [154]:
dfjj_grouped_compound_res.filter(pl.col("acc") > 0.8 ).sort("acc")
# dfjj_grouped_compound_res.filter(pl.col("COMPOUND_NAME") == '1% cholesterol + 0.25% sodium cholate' ).sort("mcc")


COMPOUND_NAME,preds,target_as_list,acc,mcc
str,list[i64],list[i64],f64,f64
"""acetazolamide""","[0, 0, … 1]","[1, 1, … 1]",0.814667,0.681183
"""terbinafine""","[0, 0, … 0]","[0, 0, … 0]",0.817667,0.635506
"""triamterene""","[1, 1, … 1]","[1, 1, … 1]",0.822,0.690274
"""allopurinol""","[1, 1, … 1]","[1, 1, … 1]",0.823,0.675775
"""amphotericin B…","[1, 1, … 0]","[1, 1, … 0]",0.829,0.698623
"""desmopressin a…","[1, 1, … 1]","[0, 0, … 1]",0.831,0.703815
"""cephalothin""","[0, 0, … 0]","[0, 0, … 0]",0.831667,0.663345
"""papaverine""","[1, 1, … 0]","[1, 1, … 0]",0.833333,0.714286
"""amitriptyline""","[1, 1, … 1]","[1, 1, … 1]",0.835333,0.66103


In [84]:
df_jj =  df.select(pl.col(["cv_index","target","COMPOUND_NAME","SACRIFICE_PERIOD","DOSE_LEVEL"])).join(df_preds.with_columns(pl.col("preds").list.len().alias("len"),(pl.col("preds").list.sum().alias("sum"))), on=["cv_index","target"],how="left")
targets = df_jj["target"].to_list()
preds = df_jj["sum"].to_list()


In [79]:
sum(hits)/ (len(hits)*250)

0.6764372759856631

In [85]:
hits = []
for target,pred in zip(targets,preds):
    if target == 1:
        hit = pred
    else:
        
        hit = 250-pred
    hits.append(hit)
df_hits = pl.DataFrame({"cv_index":df_jj["cv_index"] , "hits":hits})
df_jjj = df_jj.join(df_hits, on="cv_index",how="left")

In [109]:
df_dose_grouped = df_jjj.groupby(["DOSE_LEVEL"]).agg(pl.sum("target").alias("num_lesions"),pl.sum("hits").alias("hits"),
                                     pl.col("hits").len().alias("n")*250).with_columns((pl.col("hits")/pl.col("n")).alias("acc")).sort("acc")
df_dose_grouped

C:\Users\Tommaso\AppData\Local\Temp\ipykernel_5992\502694499.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  df_dose_grouped = df_jjj.groupby(["DOSE_LEVEL"]).agg(pl.sum("target").alias("num_lesions"),pl.sum("hits").alias("hits"),


DOSE_LEVEL,num_lesions,hits,n,acc
str,i32,i64,u32,f64
"""Middle""",277,85647,140000,0.611764
"""Low""",170,97461,140000,0.69615
"""High""",399,99981,138500,0.721884


In [110]:
df_period_grouped = df_jjj.groupby(["SACRIFICE_PERIOD"]).agg(pl.sum("target").alias("num_lesions"),pl.sum("hits").alias("hits"),
                                        pl.col("hits").len().alias("n")*250).with_columns((pl.col("hits")/pl.col("n")).alias("acc")).sort("acc")
df_period_grouped

C:\Users\Tommaso\AppData\Local\Temp\ipykernel_5992\3605673169.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  df_period_grouped = df_jjj.groupby(["SACRIFICE_PERIOD"]).agg(pl.sum("target").alias("num_lesions"),pl.sum("hits").alias("hits"),


SACRIFICE_PERIOD,num_lesions,hits,n,acc
str,i32,i64,u32,f64
"""4 day""",157,70145,105500,0.664882
"""15 day""",229,70771,105000,0.67401
"""8 day""",196,71269,105250,0.67714
"""29 day""",264,70904,102750,0.690063


In [171]:
df_comp_grouped = df_jjj.group_by("COMPOUND_NAME").agg(pl.sum("target").alias("num_lesions"),pl.sum("hits").alias("hits"),
                                     pl.col("hits").len().alias("n")*250).with_columns((pl.col("hits")/pl.col("n")).alias("acc")).sort("acc")
df_comp_grouped.filter(pl.col("acc")>0.80)

COMPOUND_NAME,num_lesions,hits,n,acc
str,i32,i64,u32,f64
"""acetazolamide""",8,2444,3000,0.814667
"""terbinafine""",6,2453,3000,0.817667
"""triamterene""",8,2466,3000,0.822
"""allopurinol""",8,2469,3000,0.823
"""amphotericin B…",8,2487,3000,0.829
"""desmopressin a…",4,2493,3000,0.831
"""cephalothin""",6,2495,3000,0.831667
"""papaverine""",7,2500,3000,0.833333
"""amitriptyline""",5,2506,3000,0.835333


In [65]:
df_preds.with_columns(pl.col("preds").list.len().alias("len"),(pl.col("preds").list.sum().alias("sum")))

cv_index,target,preds,len,sum
u32,i32,list[i64],u32,i64
0,1,"[1, 1, … 1]",250,250
1,0,"[1, 1, … 1]",250,250
2,1,"[1, 1, … 1]",250,250
3,0,"[1, 1, … 1]",250,250
4,1,"[1, 1, … 1]",250,250
5,0,"[1, 1, … 1]",250,250
6,1,"[1, 1, … 1]",250,250
7,0,"[1, 1, … 1]",250,250
8,1,"[1, 1, … 1]",250,250


In [46]:
df_temp

cv_index,target,COMPOUND_NAME,SACRIFICE_PERIOD,DOSE_LEVEL,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_1,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_2,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_3,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_4,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_1_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_1_fold_1,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_1_fold_2,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_1_fold_3,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_1_fold_4,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_1,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_2,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_3,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_4,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_1,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_2,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_3,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_4,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_4_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_4_fold_1,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_4_fold_2,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_4_fold_3,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_4_fold_4,outer_purged_COMPOUND_NAME_replica_0_fold_1_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_1_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_1,outer_purged_COMPOUND_NAME_replica_0_fold_1_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_2,outer_purged_COMPOUND_NAME_replica_0_fold_1_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_3,outer_purged_COMPOUND_NAME_replica_0_fold_1_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_4,outer_purged_COMPOUND_NAME_replica_0_fold_1_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_1_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_1_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_1_fold_1,…,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_3_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,outer_purged_COM

In [43]:
pred_cols_by_model

[('outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0',
  'outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred'),
 ('outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_1',
  'outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_1_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred'),
 ('outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_2',
  'outer_purged_CO

In [40]:
df_temp

NameError: name 'df_temp' is not defined

In [38]:
df

cv_index,COMPOUND_NAME,SACRIFICE_PERIOD,DOSE_LEVEL,num_samples,lesion_prob,weight_mean,SMILES_CODE_MoLFormer-XL-both-10pct_embeddings,COMPOUND_NAME_text-embedding-ada-002_embeddings,SMILES_CODE_text-embedding-ada-002_embeddings,SACRIFICE_PERIOD_text-embedding-ada-002_embeddings,DOSE_LEVEL_text-embedding-ada-002_embeddings,DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_LEVEL_text-embedding-ada-002_embeddings,COMPOUND_NAME_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,COMPOUND_NAME_SMILES_CODE_SACRIFICE_PERIOD_DOSE_DOSE_UNIT_text-embedding-ada-002_embeddings,COMPOUND_NAME_text-embedding-3-small_embeddings,SMILES_CODE_text-embedding-3-small_embeddings,SACRIFICE_PERIOD_text-embedding-3-small_embeddings,DOSE_LEVEL_text-embedding-3-small_embeddings,DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_DOSE_LEVEL_text-embedding-3-small_embeddings,SMILES_CODE_DOSE_LEVEL_text-embedding-3-small_embeddings,COMPOUND_NAME_SMILES_CODE_DOSE_LEVEL_text-embedding-3-small_embeddings,COMPOUND_NAME_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_SMILES_CODE_DOSE_DOSE_UNIT_text-embedding-3-small_embeddings,COMPOUND_NAME_text-embedding-3-large_embeddings,SMILES_CODE_text-embedding-3-large_embeddings,SACRIFICE_PERIOD_text-embedding-3-large_embeddings,DOSE_LEVEL_text-embedding-3-large_embeddings,DOSE_DOSE_UNIT_text-embedding-3-large_embeddings,COMPOUND_NAME_DOSE_LEVEL_text-embedding-3-large_embeddings,SMILES_CODE_DOSE_LEVEL_text-embedding-3-large_embeddings,…,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_3_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_2_fold_4_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_0_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_1_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_2_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_3_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,outer_purged_COMPOUND_NAME_replica_9_fold_3_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_3_fold_4_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred,o

In [37]:
for (input_name,model,hp_name,pred_cols_by_model) in predictions_by_model:
    print(pred_cols_by_model)


[('outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0', 'outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred'), ('outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_1', 'outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_1_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred'), ('outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_2', 'outer_purged_COMPOUND_N

In [32]:
df.select(pl.col(["cv_index","target",outs[0][0],outs[0][1]]))

cv_index,target,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0,outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred
u32,i32,str,i32
0,1,"""test""",1
1,0,"""train""",0
2,1,"""train""",1
3,0,"""train""",0
4,1,"""test""",1
5,0,"""train""",0
6,1,"""train""",1
7,0,"""train""",0
8,1,"""train""",1


In [33]:
outs[0]

('outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0',
 'outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred')

In [34]:
pred_cols[0]

'outer_purged_COMPOUND_NAME_replica_0_fold_0_inner_stratified_DOSE_LEVEL_SACRIFICE_PERIOD_target_replica_0_fold_0_COMPOUND_NAME_text-embedding-3-large_embeddings_SACRIFICE_PERIOD_text-embedding-3-large_embeddings_DOSE_LEVEL_text-embedding-3-large_embeddings_RandomForest_n_estimators_1000_max_depth_15_y_pred'